In [271]:
import pandas as pd
import utils 
import plotly.graph_objects as go
import insturment

In [272]:
pair = "GBP_JPY"
granularity = "H1"
ma_list =[16,64]
i_pair = insturment.Instrument.get_instrument_by_name(pair)

In [273]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
# df.head()
# df.describe()
none_columns = ['time', 'volume']
mod_columns = [x for x in df.columns if x not in none_columns ]
df[mod_columns] = df[mod_columns].apply(pd.to_numeric)

In [274]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()
# df_ma.head()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True,inplace=True )

In [275]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2024-02-12T19:00:00.000000000Z,188.582,188.654,188.566,188.622,188.384750,188.363328
1,2024-02-12T20:00:00.000000000Z,188.623,188.642,188.558,188.594,188.393750,188.380141
2,2024-02-12T21:00:00.000000000Z,188.596,188.622,188.566,188.614,188.404250,188.394906
3,2024-02-12T22:00:00.000000000Z,188.612,188.630,188.536,188.554,188.409750,188.405219
4,2024-02-12T23:00:00.000000000Z,188.534,188.652,188.510,188.650,188.415312,188.418172


In [276]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

the top indcates a buy if row.DIFF >= 0 and row.DIFF_PREV < 0:
 and bottom one is a sell  if row.DIFF <= 0 and row.DIFF_PREV > 0:

In [277]:
df_ma['DIFF'] =df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE != 0].copy()

making a new column, taking exsitsing dataFrame and useing Apply{applies the the whole date frame} axis 1 mean we are working row by row, the result of the fuction is stored in the 'IS_TRADE' column.

shifts the colums down by one ...-1 will shift it up 

a fuction to calculate the a value for a nnew column

In [278]:
df_trades


,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
4,2024-02-12T23:00:00.000000000Z,188.534,188.652,188.510,188.650,188.415312,188.418172,-0.002859,0.004531,-1
5,2024-02-13T00:00:00.000000000Z,188.650,188.682,188.492,188.500,188.430062,188.429516,0.000547,-0.002859,1
54,2024-02-15T01:00:00.000000000Z,188.914,188.930,188.626,188.692,189.063312,189.076953,-0.013641,0.006625,-1
89,2024-02-16T12:00:00.000000000Z,189.216,189.250,189.086,189.110,189.036312,189.025359,0.010953,-0.013703,1
258,2024-02-27T13:00:00.000000000Z,190.494,190.581,190.372,190.578,190.770625,190.797313,-0.026688,0.011391,-1
...,...,...,...,...,...,...,...,...,...,...
3498,2024-09-03T14:00:00.000000000Z,190.892,191.081,190.322,190.616,191.902813,191.952625,-0.049813,0.102656,-1
3669,2024-09-12T17:00:00.000000000Z,186.590,186.656,186.334,186.384,186.083812,186.052141,0.031672,-0.018547,1
3684,2024-09-13T08:00:00.000000000Z,185.250,185.313,184.572,184.932,185.681125,185.706141,-0.025016,0.059359,-1
3722,2024-09-16T22:00:00.000000000Z,185.876,186.068,185.796,186.064,185.377312,185.334891,0.042422,-0.074094,1


In [279]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"]  * df_trades["IS_TRADE"]

In [280]:
df_trades["GAIN"].sum()

np.float64(1148.400000000015)

In [281]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
4,2024-02-12T23:00:00.000000000Z,188.534,188.652,188.510,188.650,188.415312,188.418172,-0.002859,0.004531,-1,-15.0,15.0
5,2024-02-13T00:00:00.000000000Z,188.650,188.682,188.492,188.500,188.430062,188.429516,0.000547,-0.002859,1,19.2,19.2
54,2024-02-15T01:00:00.000000000Z,188.914,188.930,188.626,188.692,189.063312,189.076953,-0.013641,0.006625,-1,41.8,-41.8
89,2024-02-16T12:00:00.000000000Z,189.216,189.250,189.086,189.110,189.036312,189.025359,0.010953,-0.013703,1,146.8,146.8
258,2024-02-27T13:00:00.000000000Z,190.494,190.581,190.372,190.578,190.770625,190.797313,-0.026688,0.011391,-1,-54.2,54.2


In [282]:
df_plot = df_ma.iloc[200:350].copy()

In [283]:
df_plot.shape


(150, 10)

In [284]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x= df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
     line=dict(width=1),opacity=1,
     increasing_fillcolor='#24A06B',
     decreasing_fillcolor='#CC2E3C',
     increasing_line_color='#2EC886',
     decreasing_line_color='#FF3A4C'))
     
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col))

fig.update_layout(width=1000,height=600,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor= "#1e1e1e",
    plot_bgcolor="#1e1e1e")

fig.update_xaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True,rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True)

fig.show()